### Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from keras.datasets import imdb
import tensorflow_datasets as tfds
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Dense, Dropout, MaxPooling2D
from keras.applications.vgg16 import VGG16
from keras import regularizers
import numpy as np
import matplotlib.pyplot as plt
import random

c:\users\danie\pycharmprojects\tensorflowtest\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\danie\pycharmprojects\tensorflowtest\venv\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\users\danie\pycharmprojects\tensorflowtest\venv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Import Images


In [20]:
train_dir = "./Data/Train"
test_dir = "./Data/Test"
val_dir = "./Data/Val"

train_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

batch_size = 20

In [21]:
train_generator = train_datagen.flow_from_directory(directory=train_dir, target_size=(150, 150), batch_size=batch_size,
                                                    class_mode="binary")
test_generator = test_datagen.flow_from_directory(directory=test_dir, target_size=(150, 150), batch_size=batch_size,
                                                  class_mode="binary")
val_generator = val_datagen.flow_from_directory(directory=val_dir, target_size=(150, 150), batch_size=batch_size,
                                                class_mode="binary")

Found 19999 images belonging to 2 classes.
Found 2499 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


## Create Model
Begin with the convolutional base (the weights/training come from imagenet, the convolutional base is more generic, so we won't use the Densely Connected Classifier and as such make include_top = False. Finally, we define our input shape, but we don't always need to as convolutional networks can process inputs of any size)

In [22]:
conv_base = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))

In [16]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

## Feature Extraction
We essentially get the convolutional base's output for each batch in the generator, and store it into a numpy array (we need to make sure to flatten the numpy array afterwards tho so that each element has one value)

In [31]:
def extract_features(generator, batch_count):
    features = np.zeros(shape=(batch_count, batch_size, 4, 4, 512))
    i = 0
    labels = np.zeros(shape=(batch_count, batch_size))
    for image_batch, label_batch in generator:
        output_batch = conv_base.predict(image_batch)
        features[i] = output_batch
        labels[i] = label_batch
        i += 1
        if i >= batch_count:
            break
    features.resize((batch_count * batch_size, 4, 4, 512))
    labels.resize((batch_count * batch_size))
    return features, labels

In [32]:
train_x, train_y = extract_features(train_generator, 100)
val_x, val_y = extract_features(val_generator, 50)
test_x, test_y = extract_features(test_generator, 50)


KeyboardInterrupt: 